In [25]:
import os
import subprocess

In [2]:
def download_github_repo(repo_url, target_directory=None):
    if target_directory is None:
        target_directory = repo_url.split('/')[-1].replace('.git', '')
    
    if os.path.exists(target_directory):
        print(f"Error: Directory '{target_directory}' already exists.")
        return False
    
    try:
        subprocess.run(['git', 'clone', repo_url, target_directory], check=True)
        print(f"Successfully cloned repository to {target_directory}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error: Failed to clone repository. {e}")
        return False

In [ ]:
repo_url = 'https://github.com/karan-kr-451/Health-Care-With-Gemini.git'
download_github_repo(repo_url)

In [26]:
import os
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [3]:
def process_repository(repo_path, db_path):
    embeddings = OllamaEmbeddings(model="llama3.2:1b")
    
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    
    documents = []

    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(('.txt', '.md', '.py', '.js', '.html', '.css', '.yml', '.yaml')):
                file_path = os.path.join(root, file)
                try:
                    loader = TextLoader(file_path, encoding='utf-8')
                    documents.extend(loader.load_and_split(text_splitter))
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
    

    vectorstore = FAISS.from_documents(documents, embeddings)
    

    vectorstore.save_local(db_path)
    
    print(f"Vector database created and saved to {db_path}")

    return vectorstore

# Example usage
repo_path = "D:\Github Project\Readme Writer\Health-Care-With-Gemini"
db_path = "./vector_db"

vectorstore = process_repository(repo_path, db_path)


query = "Example"
results = vectorstore.similarity_search(query)
for doc in results:
    print(doc.page_content)

Vector database created and saved to ./vector_db
1. Install Python 3.7+ on your system.
2. Install the required packages listed in `requirements.txt`, including Streamlit, OpenAI, and PyTorch.
3. Clone or download the repository to your local machine.
4. Navigate to the project directory and run `streamlit run app.py` to start the application.

## Usage Instructions

To use this application:

1. Upload a medical report image to the Streamlit interface.
2. Select the 'gemini-1.5-flash' model and configure its settings as needed.
3. Click the "Generate Analysis" button to receive detailed analysis of the uploaded image.
4. Visualize the results alongside the original report.

## Contributing Guidelines

Contributions to this project are welcome! To contribute, please follow these guidelines:

* Fork the repository to your local machine.
* Make changes to the code and commit them with descriptive commit messages.
* Submit a pull request with your changes for review.

## License Informatio

In [ ]:
query = "streamlit"
results = vectorstore.similarity_search(query)
for doc in results:
    print(doc.page_content)

In [27]:
import os
import json
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [28]:
def analyze_repository(repo_path):
    ollama = Ollama(model="llama3.2", temperature=0.7)
    

    file_contents = {}
    file_summaries = {}

    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(('.py', '.js', '.html', '.css', '.md', '.txt', '.yml', '.yaml')):
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, repo_path)
                
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        file_contents[relative_path] = content
                        
                        # Generate summary for each file
                        summary_prompt = PromptTemplate(
                            input_variables=["content"],
                            template="Summarize the following code or text content in a few sentences:\n\n{content}"
                        )
                        summary_chain = LLMChain(llm=ollama, prompt=summary_prompt)
                        summary = summary_chain.run(content=content)
                        file_summaries[relative_path] = summary.strip()
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
    
    return file_contents, file_summaries

In [29]:
def generate_readme(repo_path, file_summaries):
    ollama = Ollama(model="llama3.2", temperature=0.7)
    
    readme_prompt = PromptTemplate(
        input_variables=["repo_name", "summaries"],
        template="""Generate a README.md file for a repository named {repo_name}.
            Use the provided summaries to create an organized and comprehensive overview of the project and its key components.

            The README should include the following sections:

            Project Title: Display the title of the project.
            Brief Description: Provide a concise overview of the project's purpose and goals in 50 to 100 words.
            Table of Contents: Help users quickly navigate to different sections of the README.
            Project Structure: Explain the organization of files and directories within the repository.
            Installation Instructions: Step-by-step instructions to set up the project locally.
            Usage Examples: Demonstrate how to use the project, including any relevant code snippets or examples.
            Contributing Guidelines: Outline how others can contribute to the project.
            License Information: Specify the license under which the project is released.
            Contact Information: Provide details for users to reach out for support or inquiries.
            Use the following file summaries to inform the structure and details of the README:
            {summaries}""")
    
    readme_chain = LLMChain(llm=ollama, prompt=readme_prompt)
    repo_name = os.path.basename(repo_path)
    summaries_text = json.dumps(file_summaries, indent=2)
    readme_content = readme_chain.run(repo_name=repo_name, summaries=summaries_text)
    
    with open(os.path.join(repo_path, 'README.md'), 'w', encoding='utf-8') as f:
        f.write(readme_content)
    
    print("README.md generated successfully.")

In [30]:
def generate_code_documentation(repo_path, file_contents):
    ollama = Ollama(model="llama3.2",temperature=0.9)
    
    doc_prompt = PromptTemplate(
        input_variables=["file_path", "content"],
        template="Generate detailed documentation for the following code file. Include an overview of the file's purpose, any classes or functions defined, and important logic or algorithms:\n\nFile: {file_path}\n\nContent:\n{content}"
    )
    
    doc_chain = LLMChain(llm=ollama, prompt=doc_prompt)
    
    documentation = {}
    for file_path, content in file_contents.items():
        if file_path.endswith(('.py', '.js')):
            doc = doc_chain.run(file_path=file_path, content=content)
            documentation[file_path] = doc.strip()
    
    doc_file_path = os.path.join(repo_path, 'CODE_DOCUMENTATION.md')
    with open(doc_file_path, 'w', encoding='utf-8') as f:
        for file_path, doc in documentation.items():
            f.write(f"# {file_path}\n\n{doc}\n\n---\n\n")
    
    print("CODE_DOCUMENTATION.md generated successfully.")

In [31]:
def chat_with_code(repo_path, file_contents):
    ollama = Ollama(model="llama3.2", temperature=0.7)
    memory = ConversationBufferMemory(return_messages=True, input_key="human_input")
    
    chat_prompt = PromptTemplate(
        input_variables=["history", "human_input", "repo_content"],
        template="You are an AI assistant specialized in code analysis, improvement, and bug fixing. Use the following repository content as context:\n\n{repo_content}\n\nConversation history:\n{history}\nHuman: {human_input}\nAI:"
    )
    
    chat_chain = LLMChain(llm=ollama, prompt=chat_prompt, memory=memory)
    
    repo_content = json.dumps(file_contents, indent=2)
    
    print("Chat with your code assistant. Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        
        response = chat_chain({"human_input": user_input, "repo_content": repo_content})
        print("AI:", response['text'])

In [32]:
# Main execution
repo_path = "D:\Github Project\Readme Writer\Health-Care-With-Gemini"

file_contents, file_summaries = analyze_repository(repo_path)

In [14]:
generate_readme(repo_path, file_summaries)

README.md generated successfully.


In [10]:
generate_code_documentation(repo_path, file_contents)

CODE_DOCUMENTATION.md generated successfully.


In [ ]:
chat_with_code(repo_path, file_contents)

In [ ]:
def chat_with_code(repo_path, file_contents):
    ollama = Ollama(model="llama3.2", temperature=0.7)
    memory = ConversationBufferMemory(return_messages=True, input_key="human_input")
    
    chat_prompt = PromptTemplate(
        input_variables=["history", "human_input", "repo_content"],
        template="You are an AI assistant specialized in code analysis, improvement, and bug fixing. Use the following repository content as context:\n\n{repo_content}\n\nConversation history:\n{history}\nHuman: {human_input}\nAI:"
    )
    
    chat_chain = LLMChain(llm=ollama, prompt=chat_prompt, memory=memory)
    
    repo_content = json.dumps(file_contents, indent=2)
    
    print("Chat with your code assistant. Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        
        response = chat_chain({"human_input": user_input, "repo_content": repo_content})
        print("AI:", response['text'])

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [39]:
from concurrent.futures import ThreadPoolExecutor
import time
import hashlib

def solveBot(file_contents):
    # Initialize the Ollama model (assuming it works similarly to OpenAI models)
    ollama = Ollama(model='llama3.2', temperature=0.7)
    
    # Set up memory for conversation (limiting to the last 5 exchanges)
    memory = ConversationBufferMemory(return_messages=True)

    # System message that sets the context for the AI assistant
    system_message = f"You are an AI assistant specialized in code analysis, improvement, and bug fixing. Use the following repository content as context:\n\n{file_contents}\n\n"
    
    # Chat prompt template for the conversation
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),   # This expects user input
        ("ai", "")  # AI response
    ])

    # Set up the chain with the language model and the prompt template
    chain = LLMChain(llm=ollama, prompt=chat_prompt, memory=memory)

    # Initialize a response cache for performance improvement
    response_cache = {}

    # Function to create a cache key based on user input and conversation history
    def get_cache_key(user_input):
        history = str(memory.chat_memory.messages)  # Get the conversation history
        return hashlib.md5((user_input + history).encode()).hexdigest()

    # Function to get AI response, using caching for repeated questions
    def get_ai_response(user_input):
        cache_key = get_cache_key(user_input)
        
        # Check if the response is already in cache
        if cache_key in response_cache:
            print("(Cached response)")
            return response_cache[cache_key]
        
        # Invoke the language model chain with user input
        response = chain.invoke({"input": user_input})  # Ensure "input" matches the template
        
        # Cache the response for future use
        response_cache[cache_key] = response['text'] if 'text' in response else response
        
        return response_cache[cache_key]

    # Set up the ThreadPoolExecutor to handle asynchronous response generation
    executor = ThreadPoolExecutor(max_workers=1)

    print("Chat with your code assistant. Type 'exit' to end the conversation.")
    
    # Main conversation loop
    while True:
        user_input = input("You: ")  # Get user input
        if user_input.lower() == 'exit':  # Exit condition
            break
        
        # Submit the AI response task to the executor for asynchronous execution
        future = executor.submit(get_ai_response, user_input)
        
        # Simulate thinking while AI is processing the response
        print("AI is thinking...", end="", flush=True)
        while not future.done():
            time.sleep(0.5)
            print(".", end="", flush=True)
        
        # Once AI is done, print the response
        print("\nAI:", future.result())

    # Shut down the executor when the conversation ends
    executor.shutdown()

# Example Usage:
# file_contents = analyze_repository(repo_path)  # Analyze repo contents (placeholder for now)
# solveBot(file_contents)


In [40]:
solveBot(file_contents)

Chat with your code assistant. Type 'exit' to end the conversation.
AI is thinking...

ValueError: Missing some input keys: {"'app.py'"}

In [36]:
print(file_contents)

{'app.py': 'import google.generativeai as genai\nfrom dotenv import load_dotenv\nimport os\nimport streamlit as st\nfrom src.helper import *\n\n\nload_dotenv()\ngenai.configure(api_key=os.getenv("API_KEY"))\n\nmodel = genai.GenerativeModel(model_name=\'gemini-1.5-flash\',\n                              generation_config=generation_config,\n                              safety_settings=safety_settings)\n\n\nst.set_page_config(page_title="Health Care Assistant", page_icon="🩺", \nlayout="wide")\nst.title("Health Care Assistant 👨\u200d⚕️ 🩺 🏥")\nst.subheader("An app to help with medical analysis using images")\n\nfile_uploaded = st.file_uploader(\'Upload the image for Analysis\', \ntype=[\'png\',\'jpg\',\'jpeg\'])\n\nif file_uploaded:\n    st.image(file_uploaded, width=200, caption=\'Uploaded Image\')\n    \nsubmit=st.button("Generate Analysis")\n\nif submit:\n\n    image_data = file_uploaded.getvalue()\n    \n    image_parts = [\n        {\n            "mime_type" : "image/jpg",\n         